In [5]:
import numpy as np
import pandas as pd
import os
import pickle
import sys
import scipy.stats as stats
import matplotlib.pylab as plt
%matplotlib inline

In [22]:
os.getcwd()
os.listdir()

['.Rhistory',
 'js1_bi.csv',
 'Bipartite_graph.ipynb',
 'Junction_Sequences_NTRK1_nonCRC_4_19_20_deid.xlsx',
 'Table1.rds',
 'nonCRC_Table1.pkl',
 'Fusion_Gene_ NTRK1_partners.ipynb',
 'Fusion_Gene_nonCRC.ipynb',
 'Junction_Sequences_CRC_4_19_20_deid.xlsx',
 'js1_count.pkl',
 '.RData',
 'Table1.pkl',
 '.ipynb_checkpoints',
 'Fusion_Gene_R.ipynb',
 'Fusion_Gene_CRC.ipynb',
 'Fusion_Gene_R.R']

In [6]:
df = pd.read_excel('Junction_Sequences_CRC_4_19_20_deid.xlsx')
df = df[df.MSIstatus != 'MSI ambiguous']
NTRK1_CRC = df[df.gene1_RE == 'NTRK1']

NTRK1_nonCRC = pd.read_excel('Junction_Sequences_NTRK1_nonCRC_4_19_20_deid.xlsx')
NTRK1_nonCRC = NTRK1_nonCRC[NTRK1_nonCRC.MSIstatus != 'MSI ambiguous']

In [7]:
def cound_mononucleotide(i):
    A_ct = i.count('A') + i.count('a')
    T_ct = i.count('T') + i.count('t')
    C_ct = i.count('C') + i.count('c')
    G_ct = i.count('G') + i.count('g')
    tmp = {'A':A_ct, 'T':T_ct, 'C':C_ct,'G':G_ct}
    return tmp
def count_mono_series(indexed):
    save = {}
    for i in range(len(indexed)):
        save[indexed.index[i]] = cound_mononucleotide(indexed[i])
    return save

def cound_binucleotide(str):
    list = ['A','T','C','G']
    tmp = {}
    for i in list:
        for j in list:
            key = i+j
            tmp[key] = str.count(key)
    return tmp
def count_bi_series(indexed):
    save = {}
    for i in range(len(indexed)):
        save[indexed.index[i]] = cound_binucleotide(indexed[i])
    return save

def my_referse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return "".join(complement.get(base, base) for base in reversed(seq))


def combine_rc_percentage(dg):
    df = dg.copy()
    df['count'] = df.sum(axis=1)
    cols_rest = list(df.columns)
    cols_1 = []
    cols_2 = []
    for i in df.columns:
        reverse_tmp = my_referse_complement(i)
        if reverse_tmp in list(df.columns):
            cols_rest.remove(i)
            if reverse_tmp not in cols_1:
                cols_1.append(i)
                cols_2.append(reverse_tmp)   
    dt = pd.DataFrame()
    #cols_c = []
    for i,j in zip(cols_1,cols_2):
        if i==j:
            #cols_c.append(i)
            dt[i] = df[i]/df['count']
        else:
            k = i+"+"+j
            #cols_c.append(k)
            dt[k] = (df[i]+df[j])/df['count']           
    dt = pd.concat([dt, df[cols_rest]], axis=1)
    return dt

def mean_mannwhitneyu(df):
    gb = df.groupby('MSIstatus')
    tmp = gb.agg('mean').T # compute mean
    tmp.rename(columns={"MSI-H":'MSI_mean',"MSS":'MSS_mean'}, inplace = True)
    msi = gb.get_group('MSI-H')
    mss = gb.get_group('MSS')
    p_list = []
    for i in tmp.index:
        stat, pval = stats.mannwhitneyu(msi[i], mss[i]) # compute p value
        p_list.append(pval)
    tmp['MW_2tailed_pval'] = p_list
    return tmp

# CRC

## JS1

In [17]:
NTRK1_CRC_js1 = NTRK1_CRC[['Junction Sequence 1','MSIstatus']]
NTRK1_CRC_js1.index = NTRK1_CRC['deidentifiedSpecimenName']
NTRK1_CRC_js1.shape

(45, 2)

In [9]:
NTRK1_CRC_js1_mono_count = count_mono_series(NTRK1_CRC_js1.iloc[:,0])
NTRK1_CRC_js1_mono_df = pd.concat([pd.DataFrame(NTRK1_CRC_js1_mono_count).T, NTRK1_CRC_js1['MSIstatus']],\
                                   axis=1, sort=False)
#NTRK1_CRC_js1_mono_df.head()
#combine_rc_table2(NTRK1_CRC_js1_mono_df).head()
combine_rc_percentage(NTRK1_CRC_js1_mono_df).head()

,A+T,C+G,MSIstatus,count
deidentifiedSpecimenName,,,,
XRN:AFAEVN,0.465347,0.534653,MSI-H,101
XRN:MPPMPA,0.485149,0.514851,MSI-H,101
XRN:30NP36,0.450000,0.550000,MSI-H,100
XRN:JPE87T,0.400000,0.600000,MSI-H,100
XRN:XQRHSE,0.390000,0.610000,MSI-H,100


In [10]:
NTRK1_CRC_js1_bi_count = count_bi_series(NTRK1_CRC_js1.iloc[:,0])
NTRK1_CRC_js1_bi_df = pd.concat([pd.DataFrame(NTRK1_CRC_js1_bi_count).T, NTRK1_CRC_js1['MSIstatus']],\
                                   axis=1, sort=False)
#NTRK1_CRC_js1_bi_df.head()
#combine_rc_percentage(NTRK1_CRC_js1_bi_df).head()

In [11]:
mean_mannwhitneyu(combine_rc_percentage(NTRK1_CRC_js1_mono_df))

MSIstatus,MSI_mean,MSS_mean,MW_2tailed_pval
A+T,0.460412,0.492507,0.114573
C+G,0.539588,0.507493,0.114573
count,101.179487,101.166667,0.078806


In [12]:
mean_mannwhitneyu(combine_rc_percentage(NTRK1_CRC_js1_bi_df))

MSIstatus,MSI_mean,MSS_mean,MW_2tailed_pval
AA+TT,0.088159,0.094453,0.420594
AC+GT,0.113685,0.105974,0.338131
AG+CT,0.179945,0.177794,0.420599
AT,0.042422,0.058374,0.054422
CA+TG,0.171526,0.183669,0.290807
CC+GG,0.145500,0.127627,0.162262
CG,0.019164,0.008848,0.033795
GA+TC,0.136775,0.126746,0.420571
GC,0.068334,0.072299,0.427129
TA,0.034490,0.044217,0.158004


## JS2

In [16]:
NTRK1_CRC_js2 = NTRK1_CRC[['Junction Sequence 2','MSIstatus']]
NTRK1_CRC_js2.index = NTRK1_CRC['deidentifiedSpecimenName']
#Drop rows containing with not JS2
NTRK1_CRC_js2 = NTRK1_CRC_js2.drop(NTRK1_CRC_js2[NTRK1_CRC_js2['Junction Sequence 2'] == '-'].index) 

NTRK1_CRC_js2_mono_count = count_mono_series(NTRK1_CRC_js2.iloc[:,0])
NTRK1_CRC_js2_mono_df = pd.concat([pd.DataFrame(NTRK1_CRC_js2_mono_count).T, NTRK1_CRC_js2['MSIstatus']],\
                                   axis=1, sort=False)
NTRK1_CRC_js2_bi_count = count_bi_series(NTRK1_CRC_js2.iloc[:,0])
NTRK1_CRC_js2_bi_df = pd.concat([pd.DataFrame(NTRK1_CRC_js2_bi_count).T, NTRK1_CRC_js2['MSIstatus']],\
                                   axis=1, sort=False)
NTRK1_CRC_js2.shape

(21, 2)

In [14]:
mean_mannwhitneyu(combine_rc_percentage(NTRK1_CRC_js2_mono_df))

MSIstatus,MSI_mean,MSS_mean,MW_2tailed_pval
A+T,0.457841,0.45146,0.482123
C+G,0.542159,0.54854,0.482123
count,100.588235,100.25000,0.384325


In [15]:
mean_mannwhitneyu(combine_rc_percentage(NTRK1_CRC_js2_bi_df))

MSIstatus,MSI_mean,MSS_mean,MW_2tailed_pval
AA+TT,0.096774,0.084315,0.265284
AC+GT,0.104007,0.121604,0.236626
AG+CT,0.167836,0.179394,0.250882
AT,0.046088,0.041008,0.311150
CA+TG,0.169000,0.173481,0.310640
CC+GG,0.141014,0.138917,0.411415
CG,0.030362,0.021800,0.279784
GA+TC,0.141032,0.144470,0.482129
GC,0.075613,0.065228,0.223171
TA,0.028273,0.029783,0.393850


# nonCRC

## JS1

In [18]:
NTRK1_nonCRC_js1 = NTRK1_nonCRC[['Junction Sequence 1','MSIstatus']]
NTRK1_nonCRC_js1.index = NTRK1_nonCRC['deidentifiedSpecimenName']

NTRK1_nonCRC_js1_mono_count = count_mono_series(NTRK1_nonCRC_js1.iloc[:,0])
NTRK1_nonCRC_js1_mono_df = pd.concat([pd.DataFrame(NTRK1_nonCRC_js1_mono_count).T, NTRK1_nonCRC_js1['MSIstatus']],\
                                   axis=1, sort=False)

NTRK1_nonCRC_js1_bi_count = count_bi_series(NTRK1_nonCRC_js1.iloc[:,0])
NTRK1_nonCRC_js1_bi_df = pd.concat([pd.DataFrame(NTRK1_nonCRC_js1_bi_count).T, NTRK1_nonCRC_js1['MSIstatus']],\
                                   axis=1, sort=False)
NTRK1_nonCRC_js1.shape

(112, 2)

In [34]:
mean_mannwhitneyu(combine_rc_percentage(NTRK1_nonCRC_js1_mono_df))

MSIstatus,MSI_mean,MSS_mean,MW_2tailed_pval
A+T,0.453803,0.478503,0.291253
C+G,0.546197,0.521497,0.291253
count,105.000000,100.853211,0.120156


In [35]:
mean_mannwhitneyu(combine_rc_percentage(NTRK1_nonCRC_js1_bi_df))

MSIstatus,MSI_mean,MSS_mean,MW_2tailed_pval
AA+TT,0.048231,0.099979,0.022243
AC+GT,0.159867,0.113825,0.410876
AG+CT,0.149955,0.171723,0.258227
AT,0.051922,0.048549,0.407364
CA+TG,0.223821,0.166970,0.255323
CC+GG,0.124395,0.140390,0.249565
CG,0.016493,0.019506,0.428156
GA+TC,0.130027,0.130382,0.329406
GC,0.060263,0.066028,0.365998
TA,0.035027,0.042648,0.303687


## JS2

In [19]:
NTRK1_nonCRC_js2 = NTRK1_nonCRC[['Junction Sequence 2','MSIstatus']]
NTRK1_nonCRC_js2.index = NTRK1_nonCRC['deidentifiedSpecimenName']
#Drop rows containing with not JS2
NTRK1_nonCRC_js2 = NTRK1_nonCRC_js2.drop(NTRK1_nonCRC_js2[NTRK1_nonCRC_js2['Junction Sequence 2'] == '-'].index)

NTRK1_nonCRC_js2_mono_count = count_mono_series(NTRK1_nonCRC_js2.iloc[:,0])
NTRK1_nonCRC_js2_mono_df = pd.concat([pd.DataFrame(NTRK1_nonCRC_js2_mono_count).T, NTRK1_nonCRC_js2['MSIstatus']],\
                                   axis=1, sort=False)
NTRK1_nonCRC_js2_bi_count = count_bi_series(NTRK1_nonCRC_js2.iloc[:,0])
NTRK1_nonCRC_js2_bi_df = pd.concat([pd.DataFrame(NTRK1_nonCRC_js2_bi_count).T, NTRK1_nonCRC_js2['MSIstatus']],\
                                   axis=1, sort=False)
NTRK1_nonCRC_js2.shape

(36, 2)

In [46]:
mean_mannwhitneyu(combine_rc_percentage2(NTRK1_nonCRC_js2_mono_df))

MSIstatus,MSI_mean,MSS_mean,MW_2tailed_pval
A+T,0.477277,0.497272,0.458694
C+G,0.522723,0.502728,0.458694
count,100.500000,102.147059,0.290678


In [51]:
mean_mannwhitneyu(combine_rc_percentage(NTRK1_nonCRC_js2_bi_df))

MSIstatus,MSI_mean,MSS_mean,MW_2tailed_pval
AA+TT,0.079787,0.103496,0.213493
AC+GT,0.138298,0.126790,0.213434
AG+CT,0.196809,0.168975,0.352024
AT,0.031915,0.051832,0.142162
CA+TG,0.175532,0.172500,0.486223
CC+GG,0.095745,0.123244,0.266973
CG,0.026596,0.016061,0.430896
GA+TC,0.132979,0.124759,0.431444
GC,0.074468,0.061332,0.158210
TA,0.047872,0.051010,0.404452


In [48]:
# js1 = df[['Junction Sequence 1','MSIstatus']]
# js1.index = df['deidentifiedSpecimenName']

# js1_mono_count = count_mono_series(js1.iloc[:,0])
# js1_mono_df = pd.concat([pd.DataFrame(js1_mono_count).T, js1['MSIstatus']], axis=1)
# #js1_mono_df.head()

# js1_bi_count = count_bi_series(js1.iloc[:,0])
# js1_bi_df = pd.concat([pd.DataFrame(js1_bi_count).T, js1['MSIstatus']], axis=1)
# #js1_bi_df.head()

In [40]:
# js2 = df[['Junction Sequence 2','MSIstatus']]
# js2.index = df['deidentifiedSpecimenName']
# js2 = js2.drop(js2[js2['Junction Sequence 2'] == '-'].index) 

# js2_mono_count = count_mono_series(js2.iloc[:,0])
# js2_mono_df = pd.concat([pd.DataFrame(js2_mono_count).T, js2['MSIstatus']], axis=1)
# #js2_mono_df.head()

# js2_bi_count = count_bi_series(js2.iloc[:,0])
# js2_bi_df = pd.concat([pd.DataFrame(js2_bi_count).T, js2['MSIstatus']], axis=1)
# #js2_bi_df.head()